In [36]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import math
import lightning as L
from lightning import Trainer
from torchinfo import summary
import re
import random 

In [17]:

with open("training.txt", "r") as f:
    data = f.read().splitlines()
    data = [sentence.strip() for sentence in data]
    data = [sentence for sentence in data if len(sentence)>0]
    escape_character = re.escape(',?;.:/*!+-()[]{}"\'&')
    data = [re.sub(f"[{escape_character}]"," \g<0> ",sentence) for sentence in data]

In [27]:
from collections import Counter
vocab_size = 40000
words = [word for sentence in data for word in sentence.split(" ")]
count = Counter(words)
vocab = [word[0] for word in count.most_common(vocab_size)]
with open("vocab.txt", "w") as f:
    f.write("\n".join(vocab))

In [71]:
from sentence_dataset import SentenceDataset

In [72]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

sequence_length = 50
train_dataset = SentenceDataset(train_data, vocab, sequence_length)
test_dataset = SentenceDataset(test_data, vocab, sequence_length)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=1 )